In [1]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
train.info()
cols = train.columns.tolist()
cols = cols[0:2]+ cols[3:]
train = train[cols]
train['laterality'] = train['laterality'].apply(lambda x: 1 if x == 'R' else 0)
tmp = train[['view', 'density']]
train = pd.get_dummies(train)
train['view'] = tmp['view']
train['density'] = tmp['density']
train.describe().T
print(len(train['patient_id'].unique()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54706 entries, 0 to 54705
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   site_id                  54706 non-null  int64  
 1   patient_id               54706 non-null  int64  
 2   image_id                 54706 non-null  int64  
 3   laterality               54706 non-null  object 
 4   view                     54706 non-null  object 
 5   age                      54669 non-null  float64
 6   cancer                   54706 non-null  int64  
 7   biopsy                   54706 non-null  int64  
 8   invasive                 54706 non-null  int64  
 9   BIRADS                   26286 non-null  float64
 10  implant                  54706 non-null  int64  
 11  density                  29470 non-null  object 
 12  machine_id               54706 non-null  int64  
 13  difficult_negative_case  54706 non-null  bool   
dtypes: bool(1), float64(2)

In [18]:
def pfbeta(labels, predictions, beta=1):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = predictions[idx]
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

In [20]:
for k in train['machine_id'].unique():
    print(k, pfbeta(train['cancer'], [0 if x != k else 1 for x in train['machine_id']]))
    
for k in train['site_id'].unique():
    print(k, pfbeta(train['cancer'], [0 if x != k else 1 for x in train['site_id']]))

29 0.03416445623342175
21 0.03283932188932722
216 0.005218525766470972
93 0.009111617312072893
49 0.04974277960059951
48 0.03631936694734706
170 0.02210475732820759
210 0
190 0.007674597083653109
197 0
2 0.037502372366673
1 0.04328976105877367


In [19]:
pfbeta([1,1,1,1,0,0,0,0], [1,1,1,1,1,1,1,1], beta = 1)

0.6666666666666666

In [21]:
gb = train.groupby("machine_id").mean().sort_values("cancer", ascending = False)
display(gb)
gb = train.groupby("machine_id").count()['age']
display(gb)

,site_id,laterality,age,cancer,biopsy,invasive,BIRADS,implant,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
machine_id,,,,,,,,,,,,,,,,,,,
190,1.0,0.510345,56.296552,0.034483,0.034483,0.034483,1.024000,0.000000,0.075862,0.000000,0.475862,0.000000,0.000000,0.000000,0.524138,0.068966,0.517241,0.386207,0.027586
49,1.0,0.502954,57.268687,0.026095,0.093629,0.017638,0.820053,0.057334,0.182583,0.000765,0.488036,0.000170,0.000043,0.000213,0.510774,0.117047,0.409155,0.417570,0.054486
170,1.0,0.499458,58.984832,0.024919,0.059588,0.018418,0.827731,0.138678,0.185265,0.001083,0.471289,0.000000,0.000000,0.001083,0.526544,0.048754,0.492958,0.401950,0.047671
48,2.0,0.499023,60.435452,0.020577,0.025865,0.017013,0.000000,0.000000,0.105185,0.000000,0.490171,0.000345,0.000000,0.000000,0.509484,0.000000,0.000000,0.000000,0.000000
29,2.0,0.499698,59.153744,0.019475,0.024555,0.014757,0.000000,0.000000,0.106205,0.000000,0.499698,0.000000,0.000000,0.000000,0.500302,0.000000,0.000000,0.000000,0.000000
21,2.0,0.501520,60.198516,0.018733,0.022625,0.011921,0.000000,0.000000,0.109232,0.000000,0.492884,0.000243,0.000000,0.000000,0.506873,0.000000,0.000000,0.000000,0.000000
93,1.0,0.504961,58.162402,0.007311,0.025587,0.002611,1.071299,0.000000,0.097650,0.000000,0.454308,0.000522,0.000000,0.001044,0.544125,0.080940,0.523760,0.370235,0.025065
216,1.0,0.501048,59.490566,0.004193,0.010482,0.004193,1.028252,0.000000,0.089623,0.000000,0.483753,0.000000,0.000000,0.000000,0.516247,0.024109,0.504717,0.426101,0.045073
197,1.0,0.379310,53.206897,0.000000,0.379310,0.000000,0.250000,0.000000,0.517241,0.000000,0.482759,0.000000,0.000000,0.000000,0.517241,0.000000,0.241379,0.758621,0.000000


machine_id
21      8221
29      8267
48      8699
49     23492
93      1915
170      923
190      145
197       29
210     1070
216     1908
Name: age, dtype: int64

- machine 190 has the highest cancer rate by a pretty large margin, but only has 145 samples.
- 210 and 197 both report 0 cancer rate, 197 only has 29 samples and 210 - 1070

In [24]:
gb_age = train.groupby('age').mean().sort_values('cancer', ascending=False).reset_index()
print(f"Age/Cancer Correlation: {gb_age['age'].corr(gb_age['cancer'])}")
display(gb_age)

Age/Cancer Correlation: 0.719895773231607


,age,site_id,laterality,cancer,biopsy,invasive,BIRADS,implant,machine_id,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
0,88.0,1.108108,0.486486,0.081081,0.081081,0.081081,0.680000,0.000000,48.891892,0.135135,0.000000,0.459459,0.0,0.0,0.0,0.540541,0.162162,0.567568,0.162162,0.000000
1,74.0,1.459459,0.506757,0.076014,0.101351,0.065878,0.739130,0.013514,53.599662,0.096284,0.003378,0.476351,0.0,0.0,0.0,0.520270,0.116554,0.298986,0.111486,0.013514
2,82.0,1.241379,0.497537,0.073892,0.098522,0.044335,0.833333,0.000000,65.551724,0.083744,0.000000,0.458128,0.0,0.0,0.0,0.541872,0.024631,0.536946,0.172414,0.024631
3,80.0,1.211009,0.500000,0.064220,0.091743,0.041284,0.773438,0.000000,61.995413,0.133028,0.000000,0.458716,0.0,0.0,0.0,0.541284,0.114679,0.477064,0.178899,0.018349
4,75.0,1.398374,0.493902,0.063008,0.087398,0.038618,0.857724,0.000000,60.056911,0.073171,0.000000,0.477642,0.0,0.0,0.0,0.522358,0.111789,0.272358,0.174797,0.042683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,34.0,1.000000,0.526316,0.000000,0.000000,0.000000,0.812500,0.000000,142.000000,0.157895,0.000000,0.473684,0.0,0.0,0.0,0.526316,0.000000,0.210526,0.578947,0.210526
59,35.0,1.000000,0.555556,0.000000,0.088889,0.000000,0.906977,0.000000,49.000000,0.088889,0.000000,0.511111,0.0,0.0,0.0,0.488889,0.000000,0.000000,1.000000,0.000000
60,36.0,1.000000,0.500000,0.000000,0.125000,0.000000,0.680851,0.281250,49.000000,0.234375,0.000000,0.515625,0.0,0.0,0.0,0.484375,0.125000,0.078125,0.734375,0.062500
61,37.0,1.000000,0.473118,0.000000,0.043011,0.000000,0.684211,0.118280,96.172043,0.258065,0.000000,0.516129,0.0,0.0,0.0,0.483871,0.000000,0.150538,0.623656,0.225806


- As expected, cancer and age are correlated

In [25]:
gb_implant = train.groupby(['implant', 'site_id']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb_implant)
print(len(train.query('implant == 1 and cancer == 1')))
print(len(train.query('cancer == 1')))

,implant,site_id,laterality,age,cancer,biopsy,invasive,BIRADS,machine_id,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
0,0,1,0.502924,57.512051,0.023215,0.081307,0.015584,0.857016,73.823408,0.169068,0.000678,0.485415,0.000178,0.000036,0.000285,0.513408,0.107767,0.434848,0.403645,0.052600
1,0,2,0.500060,59.937428,0.019613,0.024378,0.014611,0.000000,32.950967,0.106841,0.000000,0.494184,0.000199,0.000000,0.000000,0.505618,0.000000,0.000000,0.000000,0.000000
2,1,1,0.501693,54.345972,0.008802,0.050779,0.008802,0.839228,59.486121,0.184834,0.000000,0.477996,0.000000,0.000000,0.000000,0.522004,0.056195,0.309411,0.579553,0.043331


13
1158


In [27]:
gb_site = train.groupby('site_id').mean().sort_values('cancer', ascending=False).reset_index()
display(gb_site)
gb_site = train.groupby('site_id').count()['age']
display(gb_site)

,site_id,laterality,age,cancer,biopsy,invasive,BIRADS,implant,machine_id,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
0,1,0.502863,57.353436,0.022494,0.079779,0.015244,0.856083,0.050036,73.106033,0.169857,0.000644,0.485044,0.000169,0.000034,0.000271,0.513839,0.105186,0.428571,0.412446,0.052136
1,2,0.500060,59.937428,0.019613,0.024378,0.014611,0.000000,0.000000,32.950967,0.106841,0.000000,0.494184,0.000199,0.000000,0.000000,0.505618,0.000000,0.000000,0.000000,0.000000


site_id
1    29482
2    25187
Name: age, dtype: int64

In [28]:
gb_lat = train.groupby(['laterality', 'site_id']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb_lat)
gb_lat = train.groupby(['laterality', 'site_id']).count()['age']
display(gb_lat)

,laterality,site_id,age,cancer,biopsy,invasive,BIRADS,implant,machine_id,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
0,0,1,57.362607,0.023373,0.080681,0.014651,0.856828,0.050153,73.152777,0.168586,0.000477,0.482317,0.000136,0.000000,0.000204,0.516865,0.105281,0.430528,0.410767,0.051721
1,1,1,57.344372,0.021625,0.078887,0.015831,0.855345,0.049919,73.059822,0.171113,0.000808,0.487739,0.000202,0.000067,0.000337,0.510846,0.105093,0.426637,0.414107,0.052546
2,1,2,59.933386,0.019770,0.024057,0.014212,0.000000,0.000000,32.929734,0.099643,0.000000,0.494641,0.000238,0.000000,0.000000,0.505121,0.000000,0.000000,0.000000,0.000000
3,0,2,59.941471,0.019457,0.024698,0.015010,0.000000,0.000000,32.972205,0.114041,0.000000,0.493726,0.000159,0.000000,0.000000,0.506115,0.000000,0.000000,0.000000,0.000000


laterality  site_id
0           1          14655
            2          12592
1           1          14827
            2          12595
Name: age, dtype: int64

In [29]:
gb = train.groupby(['view', 'site_id']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb)
display(train.groupby('view').count()['age'])

,view,site_id,laterality,age,cancer,biopsy,invasive,BIRADS,implant,machine_id,...,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
0,AT,1,0.631579,60.421053,0.105263,0.157895,0.105263,0.846154,0.000000,55.368421,...,1.0,0.0,0.0,0.0,0.0,0.0,0.263158,0.368421,0.263158,0.105263
1,MLO,1,0.499934,57.559018,0.022482,0.078125,0.014900,0.854418,0.050831,73.392867,...,0.0,0.0,0.0,0.0,0.0,1.0,0.108650,0.438489,0.402228,0.048853
2,CC,1,0.505657,57.124047,0.022419,0.081296,0.015505,0.857961,0.049309,72.831611,...,0.0,1.0,0.0,0.0,0.0,0.0,0.101341,0.418215,0.423313,0.055594
3,CC,2,0.500522,59.920222,0.019683,0.024504,0.014702,0.000000,0.000000,32.904555,...,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,MLO,2,0.499568,59.952258,0.019552,0.024264,0.014527,0.000000,0.000000,32.994660,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000
5,LM,1,0.600000,73.600000,0.000000,0.000000,0.000000,1.666667,0.000000,57.800000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.200000,0.400000,0.400000,0.000000
6,LM,2,0.600000,65.000000,0.000000,0.000000,0.000000,0.000000,0.000000,37.200000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
7,LMO,1,1.000000,49.000000,0.000000,0.000000,0.000000,1.000000,0.000000,49.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000
8,ML,1,0.625000,61.750000,0.000000,0.375000,0.000000,0.375000,0.000000,75.125000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.375000,0.625000,0.000000


view
AT        19
CC     26748
LM        10
LMO        1
ML         8
MLO    27883
Name: age, dtype: int64

In [31]:
gb_biopsy = train.groupby(['biopsy', 'site_id']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb_biopsy)
gb_biopsy = train.groupby(['biopsy', 'site_id']).count()['age']
display(gb_biopsy)

,biopsy,site_id,laterality,age,cancer,invasive,BIRADS,implant,machine_id,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
0,1,2,0.493485,61.368078,0.804560,0.599349,NaN,0.000000,33.539088,0.195440,0.000000,0.496743,0.000000,0.000000,0.000000,0.503257,0.000000,0.000000,0.000000,0.000000
1,1,1,0.497240,58.345223,0.281953,0.191083,0.000000,0.031847,55.970701,0.718047,0.001274,0.494268,0.000000,0.000000,0.001274,0.503185,0.084501,0.414862,0.445011,0.054777
2,0,1,0.503350,57.267335,0.000000,0.000000,0.950468,0.051612,74.591592,0.122331,0.000589,0.484244,0.000184,0.000037,0.000184,0.514762,0.106980,0.429760,0.409623,0.051907
3,0,2,0.500224,59.901681,0.000000,0.000000,0.000000,0.000000,32.936272,0.104627,0.000000,0.494120,0.000203,0.000000,0.000000,0.505677,0.000000,0.000000,0.000000,0.000000


biopsy  site_id
0       1          27127
        2          24573
1       1           2355
        2            614
Name: age, dtype: int64

In [34]:
gb = train.groupby(['BIRADS', 'site_id']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb)
gb = train.groupby(['BIRADS', 'site_id']).count()['age']
display(gb)
print(f"site one NAN birads: {len(train[np.isnan(train['BIRADS']) & (train['site_id'] == 1)])}")
print(f"site 2 NAN birads: {len(train[np.isnan(train['BIRADS']) & (train['site_id'] == 2)])}")

,BIRADS,site_id,laterality,age,cancer,biopsy,invasive,implant,machine_id,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
0,0.0,1,0.503875,57.058191,0.116943,0.414759,0.079253,0.050370,65.366678,0.883057,0.000704,0.486439,0.000000,0.000000,0.000881,0.511976,0.076259,0.398908,0.472526,0.051074
1,0.0,2,0.467133,55.711785,0.000000,0.000000,0.000000,0.000000,32.727343,1.000000,0.000000,0.493193,0.000389,0.000000,0.000000,0.506418,0.000000,0.000000,0.000000,0.000000
2,1.0,1,0.502156,57.042272,0.000000,0.000000,0.000000,0.055288,77.492835,0.000000,0.000444,0.484276,0.000063,0.000063,0.000190,0.514963,0.121037,0.438372,0.384606,0.054654
3,2.0,1,0.501987,59.080973,0.000000,0.000000,0.000000,0.037969,81.103311,0.000000,0.000883,0.496689,0.000883,0.000000,0.000000,0.501545,0.113466,0.452980,0.381015,0.047682


BIRADS  site_id
0.0     1           5671
        2           2571
1.0     1          15755
2.0     1           2260
Name: age, dtype: int64

site one NAN birads: 5804
site 2 NAN birads: 22616


In [36]:
gb = train.groupby(['density', 'site_id']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb)
gb = train.groupby(['density', 'site_id']).count()['age']
display(gb)

,density,site_id,laterality,age,cancer,biopsy,invasive,BIRADS,implant,machine_id,...,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
0,B,1,0.500593,59.998893,0.024425,0.077227,0.016283,0.878589,0.036124,77.100545,...,0.000553,0.473322,0.000158,0.000000,0.000237,0.525729,0.0,1.0,0.0,0.0
1,C,1,0.504887,54.787413,0.022752,0.086078,0.016263,0.810653,0.070308,72.328871,...,0.000411,0.497823,0.000164,0.000082,0.000411,0.501109,0.0,0.0,1.0,0.0
2,A,1,0.502415,59.973913,0.017069,0.064090,0.010950,0.932282,0.026731,60.804187,...,0.001610,0.467311,0.000322,0.000000,0.000000,0.530757,1.0,0.0,0.0,0.0
3,D,1,0.506823,50.436969,0.016244,0.083821,0.007797,0.855556,0.041585,71.376218,...,0.001300,0.517219,0.000000,0.000000,0.000000,0.481481,0.0,0.0,0.0,1.0


density  site_id
A        1           3105
B        1          12642
C        1          12155
D        1           1531
Name: age, dtype: int64

In [38]:
gb = train.groupby(['difficult_negative_case', 'site_id']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb)
gb = train.groupby(['difficult_negative_case', 'site_id']).count()['age']
display(gb)

,difficult_negative_case,site_id,laterality,age,cancer,biopsy,invasive,BIRADS,implant,machine_id,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
0,False,1,0.502102,57.621369,0.027097,0.027097,0.018364,1.08561,0.049133,74.468109,0.000694,0.484677,0.000204,0.000041,0.000122,0.514262,0.111202,0.436442,0.398653,0.051989
1,False,2,0.504090,60.433099,0.021959,0.021959,0.016358,NaN,0.000000,32.970128,0.000000,0.494266,0.000178,0.000000,0.000000,0.505557,0.000000,0.000000,0.000000,0.000000
2,True,1,0.506582,56.043739,0.000000,0.337256,0.000000,0.00000,0.054448,66.449142,0.000399,0.486837,0.000000,0.000000,0.000997,0.511767,0.075788,0.390108,0.479856,0.052852
3,True,2,0.466369,55.793757,0.000000,0.044593,0.000000,0.00000,0.000000,32.790784,0.000000,0.493497,0.000372,0.000000,0.000000,0.506132,0.000000,0.000000,0.000000,0.000000


difficult_negative_case  site_id
False                    1          24475
                         2          22496
True                     1           5007
                         2           2691
Name: age, dtype: int64

In [40]:
gb = train.groupby(['invasive', 'site_id']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb)
gb = train.groupby(['invasive', 'site_id']).count()['age']
display(gb)

,invasive,site_id,laterality,age,cancer,biopsy,BIRADS,implant,machine_id,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C,density_D
0,1,1,0.522222,64.828889,1.000000,1.000000,0.000000,0.028889,58.595556,0.000000,0.004444,0.493333,0.000000,0.000000,0.000000,0.502222,0.075556,0.457778,0.44000,0.026667
1,1,2,0.486413,63.307065,1.000000,1.000000,NaN,0.000000,34.510870,0.000000,0.000000,0.497283,0.000000,0.000000,0.000000,0.502717,0.000000,0.000000,0.00000,0.000000
2,0,1,0.502563,57.237565,0.007362,0.065534,0.872641,0.050363,73.330662,0.172486,0.000585,0.484915,0.000172,0.000034,0.000275,0.514018,0.105645,0.428119,0.41202,0.052530
3,0,2,0.500262,59.887465,0.005077,0.009912,0.000000,0.000000,32.927838,0.108425,0.000000,0.494138,0.000201,0.000000,0.000000,0.505661,0.000000,0.000000,0.00000,0.000000


invasive  site_id
0         1          29032
          2          24819
1         1            450
          2            368
Name: age, dtype: int64

In [41]:
trainc = train.query('site_id == 1').copy()
trainc['sdage'] = [0 if np.isnan(x) else int(x/10) for x in trainc['age'] if np.nan]
gb = trainc.groupby(['density_C', 'sdage']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb)

,density_C,sdage,site_id,laterality,age,cancer,biopsy,invasive,BIRADS,implant,...,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_D
0,1,8,1.0,0.509579,83.804598,0.099617,0.122605,0.065134,0.683417,0.000000,...,0.226054,0.003831,0.455939,0.000000,0.000000,0.000000,0.540230,0.000000,0.000000,0.000000
1,1,7,1.0,0.512104,73.471136,0.061453,0.114525,0.049348,0.852798,0.007449,...,0.156425,0.000000,0.490689,0.000000,0.000000,0.000931,0.508380,0.000000,0.000000,0.000000
2,0,7,1.0,0.498488,73.379441,0.053288,0.106198,0.032880,0.843128,0.041572,...,0.140967,0.001890,0.456916,0.000000,0.000000,0.000000,0.541194,0.197657,0.768330,0.030234
3,0,8,1.0,0.500000,83.061538,0.046154,0.100000,0.033846,0.912779,0.000000,...,0.132308,0.001538,0.461538,0.004615,0.000000,0.000000,0.532308,0.132308,0.844615,0.023077
4,1,6,1.0,0.499412,64.096357,0.031727,0.084998,0.021935,0.853014,0.073247,...,0.169604,0.000392,0.498237,0.000392,0.000000,0.000392,0.500588,0.000000,0.000000,0.000000
5,0,6,1.0,0.501045,64.461334,0.022801,0.070682,0.014820,0.922348,0.030021,...,0.131674,0.000190,0.477484,0.000000,0.000000,0.000380,0.521946,0.212046,0.754893,0.029641
6,1,5,1.0,0.507967,54.187686,0.016743,0.079665,0.011882,0.820625,0.064812,...,0.188766,0.000540,0.493654,0.000000,0.000000,0.000810,0.504996,0.000000,0.000000,0.000000
7,0,5,1.0,0.504866,54.534272,0.013253,0.063781,0.009319,0.889083,0.045144,...,0.159867,0.000621,0.479188,0.000000,0.000000,0.000207,0.519983,0.162974,0.745289,0.089252
8,1,3,1.0,0.511811,36.732283,0.011811,0.070866,0.011811,0.754808,0.114173,...,0.212598,0.000000,0.511811,0.000000,0.000000,0.000000,0.488189,0.000000,0.000000,0.000000
9,0,3,1.0,0.494624,36.731183,0.010753,0.096774,0.010753,0.970803,0.043011,...,0.139785,0.000000,0.500000,0.000000,0.000000,0.000000,0.500000,0.064516,0.564516,0.370968


In [47]:
trainB = train.query('site_id == 1').copy()
trainB['sdage'] = [0 if np.isnan(x) else int(x/10) for x in trainB['age'] if np.nan]
gb = trainB.groupby(['density_B', 'sdage']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb)

,density_B,sdage,site_id,laterality,age,cancer,biopsy,invasive,BIRADS,implant,...,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_C,density_D
0,0,8,1.0,0.513812,83.560773,0.080110,0.102210,0.055249,0.814685,0.000000,...,0.168508,0.005525,0.453039,0.002762,0.000000,0.000000,0.538674,0.237569,0.720994,0.041436
1,1,7,1.0,0.498278,73.403345,0.059026,0.107231,0.035416,0.845026,0.038367,...,0.137727,0.001476,0.455976,0.000000,0.000000,0.000000,0.542548,0.000000,0.000000,0.000000
2,0,7,1.0,0.507410,73.409010,0.051571,0.110255,0.040308,0.846915,0.023711,...,0.154713,0.001186,0.479549,0.000000,0.000000,0.000593,0.518672,0.310018,0.636633,0.047421
3,1,8,1.0,0.495446,83.085610,0.049180,0.109290,0.034608,0.869458,0.000000,...,0.153005,0.000000,0.464481,0.003643,0.000000,0.000000,0.531876,0.000000,0.000000,0.000000
4,0,6,1.0,0.501171,64.149623,0.026542,0.078324,0.016914,0.892023,0.059329,...,0.150403,0.000260,0.487900,0.000260,0.000000,0.000260,0.511319,0.290398,0.664325,0.040593
5,1,6,1.0,0.499874,64.528316,0.024918,0.072489,0.017367,0.908308,0.029449,...,0.137931,0.000252,0.480745,0.000000,0.000000,0.000503,0.518500,0.000000,0.000000,0.000000
6,1,3,1.0,0.514286,36.790476,0.019048,0.066667,0.019048,1.012195,0.000000,...,0.123810,0.000000,0.457143,0.000000,0.000000,0.000000,0.542857,0.000000,0.000000,0.000000
7,0,5,1.0,0.507196,54.202311,0.015812,0.075208,0.011352,0.844356,0.053720,...,0.179607,0.000405,0.493817,0.000000,0.000000,0.000608,0.505169,0.159538,0.750659,0.087371
8,1,5,1.0,0.504862,54.632676,0.013337,0.064462,0.009169,0.881001,0.053626,...,0.162545,0.000834,0.474021,0.000000,0.000000,0.000278,0.524868,0.000000,0.000000,0.000000
9,0,4,1.0,0.503366,44.489546,0.009922,0.080439,0.005670,0.799510,0.079731,...,0.211729,0.000886,0.502658,0.000177,0.000177,0.000000,0.496102,0.102941,0.761162,0.134302


In [48]:
trainA = train.query('site_id == 1').copy()
trainA['sdage'] = [0 if np.isnan(x) else int(x/10) for x in trainA['age'] if np.nan]
gb = trainA.groupby(['density_A', 'sdage']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb)

,density_A,sdage,site_id,laterality,age,cancer,biopsy,invasive,BIRADS,implant,...,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_B,density_C,density_D
0,0,8,1.0,0.500606,83.298182,0.064242,0.111515,0.043636,0.823339,0.000000,...,0.173333,0.001212,0.461818,0.002424,0.000000,0.000000,0.534545,0.665455,0.316364,0.018182
1,0,7,1.0,0.502033,73.450422,0.058180,0.108539,0.039099,0.849310,0.029403,...,0.144823,0.000938,0.468564,0.000000,0.000000,0.000313,0.530185,0.635909,0.335940,0.025023
2,1,7,1.0,0.504780,73.133843,0.040153,0.108987,0.028681,0.825776,0.045889,...,0.149140,0.003824,0.455067,0.000000,0.000000,0.000000,0.541109,0.000000,0.000000,0.000000
3,1,8,1.0,0.523256,83.046512,0.034884,0.058140,0.034884,1.040000,0.000000,...,0.023256,0.011628,0.441860,0.011628,0.000000,0.000000,0.534884,0.000000,0.000000,0.000000
4,0,6,1.0,0.499552,64.363433,0.027463,0.077463,0.018657,0.889343,0.046567,...,0.150597,0.000299,0.487164,0.000149,0.000000,0.000448,0.511940,0.592985,0.381045,0.023284
5,0,5,1.0,0.506908,54.372757,0.015881,0.074371,0.011104,0.846532,0.057973,...,0.178309,0.000646,0.486507,0.000000,0.000000,0.000516,0.512331,0.464687,0.478115,0.055649
6,1,4,1.0,0.495697,44.473322,0.015491,0.063683,0.006885,0.878981,0.029260,...,0.154905,0.003442,0.469880,0.000000,0.000000,0.000000,0.526678,0.000000,0.000000,0.000000
7,1,6,1.0,0.506272,64.214158,0.015233,0.062724,0.008065,0.963080,0.029570,...,0.104839,0.000000,0.466846,0.000000,0.000000,0.000000,0.533154,0.000000,0.000000,0.000000
8,0,3,1.0,0.507009,36.733645,0.011682,0.077103,0.011682,0.846154,0.086449,...,0.179907,0.000000,0.504673,0.000000,0.000000,0.000000,0.495327,0.245327,0.593458,0.161215
9,0,4,1.0,0.502283,44.596562,0.008058,0.079103,0.005238,0.808908,0.067419,...,0.208300,0.000403,0.497180,0.000134,0.000134,0.000000,0.502149,0.320038,0.576954,0.101800


In [49]:
trainD = train.query('site_id == 1').copy()
trainD['sdage'] = [0 if np.isnan(x) else int(x/10) for x in trainD['age'] if np.nan]
gb = trainD.groupby(['density_D', 'sdage']).mean().sort_values('cancer', ascending=False).reset_index()
display(gb)

,density_D,sdage,site_id,laterality,age,cancer,biopsy,invasive,BIRADS,implant,...,difficult_negative_case,view_AT,view_CC,view_LM,view_LMO,view_ML,view_MLO,density_A,density_B,density_C
0,0,8,1.0,0.502232,83.291295,0.062500,0.108259,0.043527,0.833824,0.000000,...,0.161830,0.002232,0.459821,0.003348,0.000000,0.000000,0.534598,0.095982,0.612723,0.291295
1,0,7,1.0,0.503022,73.382143,0.056868,0.109341,0.038462,0.842928,0.030220,...,0.145879,0.001374,0.465934,0.000000,0.000000,0.000275,0.532418,0.143681,0.558516,0.295055
2,1,5,1.0,0.515081,53.700696,0.030162,0.113689,0.020882,0.748588,0.037123,...,0.225058,0.000000,0.529002,0.000000,0.000000,0.000000,0.470998,0.000000,0.000000,0.000000
3,0,6,1.0,0.500653,64.345431,0.025718,0.075326,0.017493,0.899164,0.045039,...,0.143603,0.000261,0.484465,0.000131,0.000000,0.000392,0.514752,0.145692,0.518668,0.333290
4,1,6,1.0,0.493590,64.179487,0.025641,0.076923,0.000000,0.965517,0.000000,...,0.166667,0.000000,0.474359,0.000000,0.000000,0.000000,0.525641,0.000000,0.000000,0.000000
5,0,5,1.0,0.505740,54.420195,0.013949,0.068387,0.009875,0.865832,0.054561,...,0.169609,0.000617,0.483150,0.000000,0.000000,0.000494,0.515739,0.097149,0.444266,0.457104
6,0,3,1.0,0.509434,36.746631,0.013477,0.075472,0.013477,0.821549,0.078167,...,0.188679,0.000000,0.498652,0.000000,0.000000,0.000000,0.501348,0.032345,0.283019,0.684636
7,1,4,1.0,0.510554,44.497361,0.010554,0.071240,0.003958,0.863636,0.042216,...,0.154354,0.002639,0.517150,0.000000,0.000000,0.000000,0.480211,0.000000,0.000000,0.000000
8,0,4,1.0,0.500894,44.597056,0.008392,0.078690,0.005503,0.808577,0.066997,...,0.209657,0.000413,0.492915,0.000138,0.000138,0.000000,0.506397,0.079928,0.327831,0.591003
9,0,0,1.0,0.448276,NaN,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.172414,0.000000,0.448276,0.000000,0.000000,0.000000,0.551724,0.000000,0.310345,0.689655
